In [1]:
import requests as req
import json
import urllib.parse
import pandas as pd
import ast
from bs4 import BeautifulSoup
from copy import copy
import os
import shutil
from zipfile import ZipFile
import re
import glob
import tabula
import PyPDF2
from pdf2image import convert_from_path
#import textract
import csv
from pytesseract import image_to_string
from PIL import Image, ImageDraw
import numpy as np
import math
from itertools import chain
import cv2
from skimage import io
import pytesseract

In [2]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)


#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)



In [3]:
def get_boxes(img):
    h, w = img.shape
    boxes = pytesseract.image_to_boxes(img) 
    for b in boxes.splitlines():
        b = b.split(' ')
        img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)

    cv2.imshow('img', img)
    cv2.waitKey()

In [4]:
# cv2.imshow('img', words_pack_1)
# cv2.waitKey()

In [5]:
# img = proc_img('./ocr_experiments/words_pack_1.png')

# get_boxes(img)

In [11]:
def proc_img(img_path):
    
    words_pack_1 = io.imread(img_path)
    
    #words_pack_1 = remove_noise(words_pack_1)
    
    gray = get_grayscale(words_pack_1)

    words_pack_1 = thresholding(gray)

    #words_pack_1 = opening(treshholded)

    #words_pack_1 = canny(opened)
    
    return words_pack_1


def calc_qual(img_path, data_path):
    
    words_pack_origin_1 = pd.read_excel(data_path)

    words_pack_origin_1 = list(words_pack_origin_1['header_1'].apply(lambda x: re.sub('\xa0', '', x)))
    
    
    words_pack_1 = proc_img(img_path)
    
    text = image_to_string(words_pack_1, lang = 'rus').split()
    
    percent = len(list(set(text) & set(words_pack_origin_1)))/len(words_pack_origin_1)
    
    return text, percent
    

In [12]:
words_pack_1_path = './ocr_experiments/words_pack_1.png'

words_pack_origin_1 =  './ocr_experiments/words_pack_origin_1.xlsx'

In [13]:
words, perc = calc_qual(words_pack_1_path, words_pack_origin_1 )

In [14]:
perc

0.0

In [15]:
words

['пороть',
 'орераы',
 'адоуриит',
 'аАвеновия.',
 'оАвлодл',
 'ваьдавитее”',
 'одлтинау',
 'Фолллвдиие',
 'длыируфон',
 'аеауоче',
 'оиааия',
 'олемар',
 'один',
 'пиар',
 'омеротау',
 'види',
 'дру',
 'бури',
 'РАЯ',
 'сори',
 'им']